In [2]:
import itertools
from pprint import pprint
from qiskit import QuantumCircuit, Aer, execute
backend = Aer.get_backend('qasm_simulator')  

maxgc = 4
gs = ['x','ccx']
qbnos = 4
qb = list(range(0,qbnos))

C:\Users\DELL\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\DELL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.jpijnswnnan3ce6lli5fwsphut2vxmth.gfortran-win_amd64.dll
C:\Users\DELL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
qb

[0, 1, 2, 3]

In [4]:
def toStr(n,base):
    '''
    Convert a decimal number to base-n
    '''
    convertString = "0123456789ABCDEF"
    if n < base:
        return convertString[n]
    else:
        return toStr(n//base,base) + convertString[n%base]

In [5]:
toStr(15,16)

'F'

In [6]:
def X_choice(opcode):
    c = list(itertools.combinations(qb, 1))
    #print(c)
    allc = []
    for i in c:
        allc.append(opcode+str(i[0]))
    return allc

In [7]:
def CCX_choice(opcode):
    c = list(itertools.combinations(qb, 3))
    allc = []
    #print(c)
    for i in c:
        allc.append(opcode+str(i[0])+str(i[1])+str(i[2]))
        allc.append(opcode+str(i[1])+str(i[2])+str(i[0]))
        allc.append(opcode+str(i[2])+str(i[0])+str(i[1]))
    return allc

In [8]:
setX = X_choice('0')
setCCX = CCX_choice('1')

In [9]:
setX

['00', '01', '02', '03']

In [10]:
setCCX

['1012',
 '1120',
 '1201',
 '1013',
 '1130',
 '1301',
 '1023',
 '1230',
 '1302',
 '1123',
 '1231',
 '1312']

In [11]:
def progs(gc):
    p = []
    for i in range(0,len(gs)**gc):
        gseq = toStr(i,len(gs)).zfill(gc)
        #print(gseq)
        cg = ['']          
        for j in range(0, len(gseq)):
            if gseq[j] == '0':
                g = list(itertools.product(cg, setX))
            if gseq[j] == '1':
                g = list(itertools.product(cg, setCCX))
            cg = []
            for k in g:
                cg.append(''.join(map(str, k)))
        #print(gseq,': ',cg)
        for j in cg:
            p.append(j)
    return p

In [12]:
ng = {}
for gc in range(1,maxgc+1):
    p = progs(gc)
    for i in p:
        op = runQprog(i)
        if op in ng:
            ng[op].append(i)
        else:
            ng[op] = [i]
    print("nGC Level",gc)
    for key, value in sorted(ng.items()):
        value.sort(key=len)
        print(key, len(value),value[0],len(value[0]))

NameError: name 'runQprog' is not defined

In [ ]:
def state_construction(index,qc):
    for i in range(4):
        if(index[i]) == '1':
            qc.x(i)

In [ ]:
possible_states = list(itertools.product('01', repeat=4))
possible_states

In [ ]:
def conv2QP(desc):
    qc_list = []
    for j in possible_states:
        qcirc = QuantumCircuit(qbnos,qbnos)
        state_construction(j,qcirc)
        i = 0
        while (i < len(desc)):
            if desc[i]=='0':
                # print('X',desc[i+1])
                qcirc.x(int(desc[i+1]))
                i+= 2
            elif desc[i]=='1':
                # print('CCX',desc[i+1],desc[i+2],desc[i+3])
                qcirc.ccx(int(desc[i+1]),int(desc[i+2]),int(desc[i+3]))
                i+= 4
        qcirc.barrier()
        for i in range(0,qbnos):
            qcirc.measure(i,i)
        qc_list.append(qcirc)
    return qc_list

In [ ]:
def runQprog(desc):
    mem_result= []
    qcirc = conv2QP(desc)
    # print(desc)
    # print(qcirc.draw())
    for i in qcirc:
        job = execute(i, backend, shots=1, memory=True)
        result = job.result()
        memory = result.get_memory()
        mem_result.append(memory[0][::-1])
    return mem_result

In [ ]:
runQprog('00')

In [ ]:
gc = 1
ng = {}
p = progs(gc)
for i in p:
    op = runQprog(i)
    ng[i] = op
print("nGC Level",gc)
# for key, value in sorted(ng.items()):
#     value.sort(key=len)
#     print(key, len(value),value[0],len(value[0]))

In [ ]:
one_gate = progs(1)

In [ ]:
l = []
for i in (one_gate):
    l.append(ng[i])

In [ ]:
l

In [ ]:
import numpy as np

In [ ]:
possible_states = ['0000',
  '0001',
  '0010',
  '0011',
  '0100',
  '0101',
  '0110',
  '0111',
  '1000',
  '1001',
  '1010',
  '1011',
  '1100',
  '1101',
  '1110',
  '1111']

In [ ]:
# short circuits at shortest nested list if table is jagged:
list(map(list, zip(*l)))

# discards no data if jagged and fills short nested lists with None
transpose_list = list(map(list, itertools.zip_longest(*l, fillvalue=None)))

In [ ]:
all_first_layer_list={}
for i in range(qbnos**2):
    all_first_layer_list[possible_states[i]] = list(np.unique(transpose_list[i]))

In [ ]:
all_first_layer_list

In [ ]:
all_first_layer_list['0000']

In [ ]:
def create_adjacency_layer(state):
    layer = 1
    first_state_adjacency = {}
    for i in possible_states:
        first_state_adjacency[i] = 0
    for i in all_first_layer_list[state]:
        if((first_state_adjacency[i] == 0)):
            first_state_adjacency[i] = 1
    for j in possible_states:
        if((first_state_adjacency[j] == 1) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 2
    for j in possible_states:
        if((first_state_adjacency[j] == 2) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 3
    for j in possible_states:
        if((first_state_adjacency[j] == 3) & (j != state)):
            for k in all_first_layer_list[j]:
                if(first_state_adjacency[k] == 0):
                    first_state_adjacency[k] = 4
    return first_state_adjacency

In [ ]:
create_adjacency_layer('0000')

In [ ]:
adjancency_matrix = []
for i in possible_states:
    adjancency_matrix.append(create_adjacency_layer(i))

In [ ]:
adjancency_matrix[0]